In [ ]:
import os, gzip, json
from mpcontribs.client import Client
from pymatgen.core import Structure
from pymatgen.ext.matproj import MPRester
from urllib.request import urlretrieve
from monty.json import MontyDecoder

In [ ]:
name = '2dmatpedia'
client = Client()
mpr = MPRester()

**Configure Project**

In [ ]:
description = """
We start from the around 80000 inorganic compounds in the Materials Project database. A geometry-based
algorithm [PRL] was used to identify layered structures among these compounds. Two-dimensional (2D)
materials were theoretically exfoliated by extracting one cluster in the standard conventional unit cell
of the layered structures screened in the above steps. A 20 Å vacuum along the c axis was imposed to
minimize the interactions of image slabs by periodic condition. Structure matcher tools from Pymatgen were
used to find duplicates of the exfoliated 2D materials. The standard workflow developed by the Materials
Project was used to perform high-throughput calculations for all the layered bulk and 2D materials screened
in this project. The calculations were performed by density functional theory as implemented in the Vienna
Ab Initio Simulation Package (VASP) software with Perdew-Burke-Ernzerhof (PBE) approximation for the
exchange-correlation functional and the frozen-core all-electron projector-augmented wave (PAW) method for
the electron-ion interaction. The cutoff energy for the plane wave expansion was set to 520 eV.
"""

legend = {
    "details": "link to detail page on 2dMatPedia",
    "source": "link to source material",
    "process": "discovery process (top-down or bottom-up)",
    "ΔE": "band gap",
    "Eᵈ": "decomposition energy",
    "Eˣ": "exfoliation energy",
    "E": "energy",
    "Eᵛᵈʷ": "van-der-Waals energy",
    "µ": "total magnetization"
}

project = {
    'is_public': True,
    'title': '2DMatPedia',
    'long_title': '2D Materials Encyclopedia',
    'owner': 'migueldiascosta@nus.edu.sg',
    'authors': 'M. Dias Costa, F.Y. Ping, Z. Jun',
    'description': description,
    'references': [
        {'label': 'WWW', 'url': 'http://www.2dmatpedia.org'},
        {'label': 'PRL', 'url': 'https://doi.org/10.1103/PhysRevLett.118.106101'}
    ]
}

# client.projects.update_entry(pk=name, project=project).result()
# client.projects.update_entry(pk=name, project={"other": legend}).result()
client.projects.update_entry(pk=name, project={"unique_identifiers": False}).result()

In [ ]:
columns = {
    'material_id': {'name': 'details'},
    'source_id': {'name': 'source'},
    'discovery_process': {'name': 'process'},
    'bandgap': {'name': 'ΔE', 'unit': 'eV'},
    'decomposition_energy': {'name': 'Eᵈ', 'unit': 'eV/atom'},
    'exfoliation_energy_per_atom': {'name': 'Eˣ', 'unit': 'eV/atom'},
    'energy_per_atom': {'name': 'E', 'unit': 'eV/atom'},
    'energy_vdw_per_atom': {'name': 'Eᵛᵈʷ', 'unit': 'eV/atom'},
    'total_magnetization': {'name': 'µ', 'unit': 'µᵇ'}    
}

init_columns = {
    v["name"]: v.get("unit")
    for v in columns.values()
}

client.init_columns(name, init_columns)
client.get_project(name).display()

**Prepare Contributions**

In [ ]:
db_json = "http://www.2dmatpedia.org/static/db.json.gz"
raw_data = []  # as read from raw files
dbfile = db_json.rsplit('/')[-1]

if not os.path.exists(dbfile):
    print('downloading', dbfile, '...')
    urlretrieve(db_json, dbfile)

with gzip.open(dbfile, 'rb') as f:
    for line in f:
        raw_data.append(json.loads(line, cls=MontyDecoder))

len(raw_data)

In [ ]:
details_url = "http://www.2dmatpedia.org/2dmaterials/doc/"
contributions = []
prefixes = {'mp', 'mvc', '2dm'}

for rd in raw_data:
    source_id = rd['source_id']
    prefix = source_id.split('-')[0]
    
    if prefix not in prefixes:
        continue
        
    identifier = rd['material_id'] if prefix == "2dm" else source_id 
    d = {}
    
    for k, col in columns.items():
        value = rd.get(k)
        if not value:
            continue
        
        unit = col.get("unit")

        if k == "material_id" or (k == "source_id" and rd[k].startswith("2dm")):
            value = f"{details_url}{rd[k]}"
        elif k == "source_id":
            value = f"https://materialsproject.org/materials/{rd[k]}"
        elif unit:
            value = f"{rd[k]} {unit}"

        d[col["name"]] = value

    contrib = {
        'project': name, 'is_public': True, 'identifier': identifier,
        'data': d, 'structures': [rd["structure"]]
    }

    contributions.append(contrib)

len(contributions)

**Submit Contributions**

In [ ]:
client.delete_contributions(name)
client.init_columns(name, init_columns)
client.submit_contributions(contributions[:10])

**Query Contributions**